# Visualizing Area Summaries

* It can be difficult to easily summarize GEE imageCollections and images over an area in an interactive environment
* geeViz's area charting methods allow for easy interactive charting with minimal additional code
* This notebook shows several examples of different ways of using the area charting methods with GEE images and imageCollections



Copyright 2025 Ian Housman

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geeviz/blob/master/examples/areaChart_examples.ipynb) 
[![github](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geeViz/blob/master/examples/areaChart_examples.ipynb)

In [ ]:
#Import modules
try:
    import  geeViz.geeView as geeView
except:
    !python -m pip install geeViz
    import  geeViz.geeView as geeView

import geeViz.getImagesLib as gil
import pandas as pd
ee = geeView.ee
Map = geeView.Map
Map.clearMap()
print('done')

* There are many different formats of data geeViz's area charting module can handle
* The most common is charting the percent or area in hectares or acres of thematic classes of an image collection
* If an imageCollection has `class_values`, `class_names`, `class_palette` properties for its images, all charts will automatically be populated with those names and colors

* First, we will look at these properties for a couple available images and image collections

In [ ]:
lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9")
nlcd = ee.ImageCollection("USGS/NLCD_RELEASES/2021_REL/NLCD").select(['landcover'])
def get_props_dataFrames(props,bandNames = None):
    props = {k:i for k,i in props.items() if k.find('_class_')>-1}

    if bandNames == None:
        bandNames = list(set([k.split('_class_')[0] for k in props.keys()]))
    out = {}
    for bn in bandNames:
        print(bn)
        df = pd.DataFrame({'Values':props[f'{bn}_class_values'],
                           'Names':props[f'{bn}_class_names'],
                           'Colors':props[f'{bn}_class_palette']})
        display(df)
        out[bn]=df
    # return out
lcms_props = lcms.first().toDictionary().getInfo()
nlcd_props = nlcd.first().toDictionary().getInfo()


lcms_thematic_bandNames = lcms.select(['Change','Land_Cover','Land_Use']).first().bandNames().getInfo()

nlcd_landcover_bandNames = ['landcover']

get_props_dataFrames(lcms_props,lcms_thematic_bandNames)
get_props_dataFrames(nlcd_props,nlcd_landcover_bandNames)


# Shorten NLCD class names for later use
nlcd_class_names_key = f'{nlcd_landcover_bandNames[0]}_class_names'
nlcd_class_names_shortened = [nm.split(':')[0] for nm in nlcd_props[nlcd_class_names_key]]
nlcd_props_shortened_names = nlcd_props
nlcd_props_shortened_names[nlcd_class_names_key] = nlcd_class_names_shortened

print('Shortened NLCD class names:')
get_props_dataFrames(nlcd_props_shortened_names,nlcd_landcover_bandNames)

## Basic Area Charting

* This example will show the most basic method for adding area chart layers

* By setting `"canAreaChart":True`, the layer will be added to area charting. Available properties (shown above) will be used to set up names and colors

* There are many options for area charting. Most of these options are provided as a dictionary using the `areaChartParams` key. This example shows two different `chartType`s 

* Using the `Map.turnOnAutoAreaCharting()` method will turn on autmatic area charting. This will use the map extent as the summary area.

* Additional methods are `Map.turnOnSelectionAreaCharting()` and `Map.turnOnUserDefinedAreaCharting()` for turning on different methods of providing areas to summarize. You can also change the method being used in the geeView UI under `Tools -> Area Tools`

In [ ]:
Map.clearMap()


Map.addLayer(lcms.select(['Land_Cover']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'chartType':'stacked-line'}},'LCMS Land Cover')
Map.addLayer(nlcd.select(['landcover']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'chartType':'stacked-bar'}},'NLCD Land Cover')



Map.turnOnAutoAreaCharting()
Map.view()

## Line and Sankey Charts

* For thematic imageCollections, line and sankey charts are supported

* You can specify one or both within the `areaChartParams` dictionary. By default, line is chosen. (e.g. `"areaChartParams":{"sankey":True}`)

* Note that sankey charts work well at showing transitions between classes. Since the number of transition classes is the number of classes<sup>2</sup>, if there are many classes, sankey charting will be quite slow and may error out.

* For sankey charting, you can specify transition periods in the code using the `sankeyTransitionPeriods` key in `areaChartParams` (e.g. `"sankeyTransitionPeriods":[[1985,1987],[2000,2002],[2020,2022]]`), or leave it blank and geeViz will try to figure out a good set given the years of the provided imageCollection. Note that if you add imageCollections with different time extents, geeViz will take the intersection for the default years. You can add and change the periods in the geeView UI as well under `Tools -> Area Tools -> Area Tools Parameters` 

In [ ]:
Map.clearMap()

Map.addLayer(lcms.select(['Change']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'sankey':True}},'LCMS Change')

Map.addLayer(lcms.select(['Land_Cover']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'sankey':True}},'LCMS Land Cover')

Map.addLayer(lcms.select(['Land_Use']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'sankey':True}},'LCMS Land Use')


Map.turnOnAutoAreaCharting()
Map.view()

*  Adding a layer to line and sankey charting is done as follows

* Any time you specify `'line':True,'sankey':True`, both line and sankey charts will be created 

* This can slow things down a lot if many layers are visible. Also, moving the map, pausing for a second or two, and then moving the map again, many times, can create a long queue and delay charting.

In [ ]:
Map.clearMap()

Map.addLayer(lcms.select(['Change']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'line':True,'sankey':True}},'LCMS Change')

Map.addLayer(lcms.select(['Land_Cover']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'line':True,'sankey':True}},'LCMS Land Cover')

Map.addLayer(lcms.select(['Land_Use']),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'line':True,'sankey':True}},'LCMS Land Use')


Map.turnOnAutoAreaCharting()
Map.view()

## Ways of adding area charting layers
* There are two methods for adding area chart layers. The first is shown above, where when adding a layer using `Map.addLayer`, specify `"canAreaChart":True`. With this method, if a layer is visible, it will be included in area charting.
* There are instances where you would like to summarize layers, but may not want them on the map or it is impossible to visualize all the thematic bands you'd like to chart. In this instance, you can use the `Map.addAreaChartLayer` method.
* If you use the `Map.addAreaChartLayer` method, you will need to use the `Map.populateAreaChartLayerSelect()` method to instantiate a selection menu for choosing which area chart layers should be charted. 
* In this example, we will summarize all thematic classes in LCMS in a single graph. This cannot be displayed on a map, but is an interesting way to look at the summarized data in charts

* Note that the dictionary of parameters is more or less the same as what you would put in the `"areaChartParams"` if you were to use the `Map.addLayer` method. 

* Note that while multi thematic band image collections can be charted in a single line chart, sankey charts can only support one band per chart. If a multi thematic band image collection is given with `"sankey":True` (as is the case in this example), separate sankey charts will be created for each band.

In [ ]:
Map.clearMap()

Map.addLayer(lcms.select(['Change_Raw_Probability.*']),{'reducer':ee.Reducer.stdDev(),'min':0,'max':10},'LCMS Change Prob')

Map.addAreaChartLayer(lcms,{'line':True},'LCMS All Thematic Classes Line',False)

Map.addAreaChartLayer(lcms,{'sankey':True},'LCMS All Thematic Classes Sankey',True)

Map.populateAreaChartLayerSelect()
Map.turnOnAutoAreaCharting()

Map.view()

## Charting Non-Thematic Data

* You can chart continuous data as well. By default, a `ee.Reducer.mean()` will be used. You can use any reducer that returns a single value per image-band (e.g. `ee.Reducer.min()`, `ee.Reducer.max()`, `ee.Reducer.stdDev()`, `ee.Reducer.mode()`, and not `ee.Reducer.percentile([0,50,100])`). 

* You can specify this using `"areaChartParams":{"reducer":ee.Reducer.mean()}`

* Optionally, you can provide a color palette to be used. Each band will be assigned to a color in the order given

* Notice in the example, the reducer for what is shown on the map is different from the zonal summary reducer. In this example, on the map the standard deviation of the probability is shown to highlight likely change, while the average over the area is shown in the chart since that is a more appropriate representation of probability over an area. 

In [ ]:
Map.clearMap()

Map.addLayer(lcms.select(['Change_Raw_Probability.*']),
             {'reducer':ee.Reducer.stdDev(),'min':0,'max':10,'canAreaChart':True,'areaChartParams':{'palette':'f39268,d54309,00a398'}},'LCMS Change Prob')


Map.turnOnAutoAreaCharting()
Map.view()

## Charting Images

* You can also chart images
* It will behave in a similar fashion to imageCollections, but will show a bar chart
* Placing names for bar charts can be challenging if the names are very long. geeViz will automatically change the bar chart to be a horizontal bar chart if long names are detected. This still does not ensure the bar charts are readable (as is the case in this example). Shortening class names is the easiest method to address this issue (shown by splitting the full NLCD landcover name with `:` and take the first part earlier in this notebook).
* If using `"autoViz":True`, be sure to copy the `_class_` properties back in


In [ ]:
Map.clearMap()


Map.addLayer(lcms.select(['Land_Cover']),{'autoViz':True,'canAreaChart':True},'LCMS Land Cover')
Map.addLayer(lcms.select(['Land_Cover']).mode().set(lcms.first().toDictionary()),{'autoViz':True,'canAreaChart':True},'LCMS Land Cover Mode')

Map.addLayer(nlcd.select(['landcover']),{'autoViz':True,'canAreaChart':True},'NLCD Land Cover')
Map.addLayer(nlcd.select(['landcover']).mode().set(nlcd.first().toDictionary()),{'autoViz':True,'canAreaChart':True},'NLCD Land Cover Mode')

# Use the shortened class names to clean up chart
Map.addLayer(nlcd.select(['landcover']).mode().set(nlcd_props_shortened_names),{'autoViz':True,'canAreaChart':True},'NLCD Land Cover Mode Shortened Names')





Map.turnOnAutoAreaCharting()
Map.view()

## Charting Images Without Color and Name Properties

* Sometimes you will have an image you add to the map with a min, max, and palette in the viz params that are thematic  or ordinal thematic data. If those images have no class_names, etc properties set, geeViz will try to render the chart properly using the given min, max, and palette if you specify `"areaChartParams":{"reducer":ee.Reducer.frequencyHistogram()}


In [ ]:

Map.clearMap()

def getMostRecentChange(c, code):
    def wrapper(img):
        yr = ee.Date(img.get("system:time_start")).get("year")
        return (
            ee.Image(yr)
            .int16()
            .rename(["year"])
            .updateMask(img.eq(code))
            .copyProperties(img, ["system:time_start"])
        )

    return c.map(wrapper)

mostRecentFastLossYear = getMostRecentChange(lcms.select(['Change']),3).max()

Map.addLayer(mostRecentFastLossYear, {'min':1985,'max':2023,'palette':["ffffe5", "fff7bc", "fee391", "fec44f", "fe9929", "ec7014", "cc4c02"],'canAreaChart':True,'areaChartParams':{'reducer':ee.Reducer.frequencyHistogram()}}, "Most Recent Fast Loss Year", True)

Map.turnOnAutoAreaCharting()
Map.view()

## Charting Time Lapses 

* Time lapses also support area charting. All functionality is the same as the `Map.addLayer` methods.

* Band names can be specified in the `areaChartParams`. If they are not specified, but `bands` is specified in the visualization parameters, those bands will be used instead. Otherwise, all bands will be shown. 


In [ ]:
Map.clearMap()

composites = ee.ImageCollection("projects/lcms-tcc-shared/assets/CONUS/Composites/Composite-Collection-yesL7")

years = list(range(1985,2024))

# Need to mosaic the tiled outputs for each year
composites = [composites.filter(ee.Filter.calendarRange(yr,yr,'year')).mosaic().set('system:time_start',ee.Date.fromYMD(yr,6,1).millis()) for yr in years]
composites = ee.ImageCollection(composites)

# Set up visualization parameters
viz = gil.vizParamsFalse10k
viz['canAreaChart']=True
viz['areaChartParams']={'bandNames':'blue,green,red,nir,swir1,swir2','palette':'00D,0D0,D00,D0D,0DD'}
Map.addTimeLapse(composites,gil.vizParamsFalse10k,'Composites')

Map.turnOnAutoAreaCharting()
Map.view()

## Charting Thematic Data without set properties

* You can chart thematic datasets that lack values, names, and palette properties by specifying the `ee.Reducer.frequencyHistogram()` as the reducer

* This is not the best method however. Charts will lack descriptive class names and colors



In [ ]:
# LCMAP example
Map.clearMap()


lcpri = ee.ImageCollection("projects/sat-io/open-datasets/LCMAP/LCPRI").select(['b1'],['LC'])


Map.addTimeLapse(lcpri,{'min':1,'max':9,'canAreaChart':True,"areaChartParams":{'reducer':ee.Reducer.frequencyHistogram()}},'LCMAP LC Primary')

Map.turnOnAutoAreaCharting()
Map.view()


### Setting properties for charting

* The easiest way to chart thematic data where each class has a number, name, and color, but lack the preset properties, is to set them on-the-fly

* These properties can then be used for charting and map rendering

In [ ]:
# LCMAP example
Map.clearMap()

lcpri_palette = ['E60000','A87000','E3E3C2','1D6330','476BA1','BAD9EB','FFFFFF','B3B0A3','A201FF']
lc_names = ['Developed','Cropland','Grass/Shrub','Tree Cover','Water','Wetlands','Ice/Snow','Barren','Class Change']
lc_numbers = list(range(1,len(lcpri_palette)+1))

lcpri = ee.ImageCollection("projects/sat-io/open-datasets/LCMAP/LCPRI").select(['b1'],['LC'])

lcpri = lcpri.map(lambda img:img.set({'LC_class_values':lc_numbers,'LC_class_names':lc_names,'LC_class_palette':lcpri_palette}))


Map.addTimeLapse(lcpri,{'autoViz':True,'canAreaChart':True,'areaChartParams':{'line':True,'sankey':True}},'LCMAP LC Primary')

Map.turnOnAutoAreaCharting()
Map.view()

## Comparing map output versions

* One common task is to understand the differences between 2 or more model runs. This can be challenging.

* This example will show three versions of LCMS products and how they relate

* This approach makes comparing the maps and their respective class counts relatively easy

* This idea could be adapted to comparing different thematic or continuous outputs

In [ ]:
Map.clearMap()

# Bring in 3 different LCMS versions
lcms_2020 = ee.ImageCollection("USFS/GTAC/LCMS/v2020-5").filter('study_area=="CONUS"')
lcms_2021 = ee.ImageCollection("USFS/GTAC/LCMS/v2021-7").filter('study_area=="CONUS"')
lcms_2022 = ee.ImageCollection("USFS/GTAC/LCMS/v2022-8").filter('study_area=="CONUS"')
lcms_2023 = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9").filter('study_area=="CONUS"')

# Choose a year to compare (any year 1985-2020)
year = 2010

# Filter off the image and set that image to the version year
lcms_2020 = lcms_2020.filter(ee.Filter.calendarRange(year,year,'year')).first().set({'year':2020,'system:time_start':ee.Date.fromYMD(2020,6,1).millis()})
lcms_2021 = lcms_2021.filter(ee.Filter.calendarRange(year,year,'year')).first().set({'year':2021,'system:time_start':ee.Date.fromYMD(2021,6,1).millis()})
lcms_2022 = lcms_2022.filter(ee.Filter.calendarRange(year,year,'year')).first().set({'year':2022,'system:time_start':ee.Date.fromYMD(2022,6,1).millis()})
lcms_2023 = lcms_2023.filter(ee.Filter.calendarRange(year,year,'year')).first().set({'year':2023,'system:time_start':ee.Date.fromYMD(2023,6,1).millis()})

# Construct the image collection
c = ee.ImageCollection([lcms_2020,lcms_2021,lcms_2022,lcms_2023])

# Add the collection as a timelapse
# Will need to specify the transition years as the years used for each image, otherwise geeViz will default to only showing the first and last year
# Note that if you specify the sankey transition periods, any periods you enter in the geeViz UI will not be used for that layer
for bn in ['Change','Land_Cover','Land_Use']:
  Map.addTimeLapse(c.select([bn]),{'autoViz':True,'canAreaChart':True,'areaChartParams':{'sankey':True,
                                                                            'sankeyTransitionPeriods':[[2020,2020],[2021,2021],[2022,2022],[2023,2023]]
                                                                            }}, f"LCMS {bn.replace('_',' ')} Comparison {year}")

Map.turnOnAutoAreaCharting()
Map.view()

## Other charting summary zone selection methods

* All examples have simply used the map extent as the zone to chart

* There are other methods available

* This example will show how to add a featureCollection to interactively select areas to summarize

* All area selection will happen in the geeViz UI, under `Tools -> Area Tools -> Select an Area on Map`

* This example also demonstrates the use of the stacked bar `chartType` and turning off the first and last MTBS burn severity classes in the chart



In [ ]:
Map.clearMap()


for bn in ['Change','Land_Cover','Land_Use']:
  Map.addLayer(lcms.select([bn]),{'autoViz':True,'canAreaChart':True}, f"LCMS {bn.replace('_',' ')}  ")


# Bring in MTBS burn boundaries
mtbsBoundaries = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")
mtbsBoundaries = mtbsBoundaries.map(
    lambda f: f.set("system:time_start", f.get("Ig_Date"))
)
# Bring in MTBS burn severity mosaics
mtbsSeverity = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1").filter(ee.Filter.stringContains('system:index','_CONUS_'))
Map.addLayer(mtbsSeverity,{'autoViz':True,'canAreaChart':True,'areaChartParams':{'chartType':'stacked-bar',"visible": [False, True, True, True, True, True, False],}}, "MTBS Burn Severity")

# For area charting you can select areas to chart a number of ways. One method is using a map layer that is selectable by clicking on each feature.
Map.addSelectLayer(
    mtbsBoundaries,
    {
        "strokeColor": "00F",
        "selectLayerNameProperty": "Incid_Name",
    },
    "MTBS Fire Boundaries"
)

Map.turnOnSelectionAreaCharting()
Map.view()
